In [1]:
# We're just importing some stuff to make sure the challenges and helper code works.
# Feel free to import anything else you need (or solve in a different language), but if you overwrite these, provided code may not work.

import pandas as pd
import numpy as np
import os
import requests
from IPython.display import Image as Ima
from IPython.display import clear_output, display
from PIL import Image
from PIL import Image as Im
from torch.utils.data import Dataset, DataLoader
import random
import torch
import joblib
import matplotlib.pyplot as plt
import json
import base64
import shutil
import warnings
warnings.filterwarnings("ignore")


dir_name = "/kaggle/input/ai-village-ctf"

# Input data files are available in the read-only "../input/" directory
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# A helper function for tracking flags. You could do it yourself, but why?

def add_flag(challenge_id: str, flag: str) -> str:
    try:
        submission_path = "/kaggle/working/submission.csv"
        template_path = f"{dir_name}/submission.csv"
        if not os.path.isfile(submission_path) :
            shutil.copyfile(template_path, submission_path)
        flags = pd.read_csv(submission_path, index_col="challenge_id")
        valid = flags.index.to_list()
        if challenge_id in valid:
            flags.at[challenge_id] = flag
            flags.to_csv(submission_path)
            print("Flag saved. If you want to update the scoreboard, click 'submit' on the right. Remember that you can only update your score on the scoreboard 50 times per day")
        else:
            print("Invalid challenge_id")
    except:
        print("We had trouble saving that flag. Keep a copy of it and contact the CTF organizers.")

In [ ]:
model = joblib.load(f"{dir_name}/crop/04JUN_crop_model.pckl")
X_comp = joblib.load(f"{dir_name}/crop/X_comp.pckl")

In [ ]:
# Helper Functions
def get_ij(idx):
    idx = 8-idx
    j = idx//3
    i = 2-idx%3
    return i, j

def resize(X, size=(224,224)):
    X_new = np.array(Im.fromarray(X.astype(np.uint8)).resize(size, resample=3))
    return X_new

def get_cropped(X, idx):
    n = 75*3
    i, j = get_ij(idx)
    crop = resize(X,(n,n))[j*75:(j+1)*75,i*75:(i+1)*75,:]
    return crop

def i2t(img:np.array):
    return np.array(img, dtype=np.float32)/255.

def make_X(x, size=(224,224)):
    side = int((len(x)//3)**0.5)
    x = np.where(x<255,x,255)
    x = np.where(x>0,x,0)
    X_np = x.astype(int).reshape([side, side, 3])
    X_np = np.array(Im.fromarray(X_np.astype(np.uint8)).resize(size, resample=Im.Resampling.NEAREST))
    return X_np

def pt_ft(X_np):
    return torch.FloatTensor(X_np)

def calc_score(actual_square, model=model, X_comp = np.divide(X_comp,0b1010)):
    expected = ((25.5-X_comp)).flatten()
    actual = actual_square.flatten().astype(int)
    sse = ((actual-(expected*10).astype(int))**2).sum()
    redness = (actual_square[:,:,0][actual_square[:,:,0]>230]**2).sum()+1
    return sse*redness

def score(x, given_idx=-1, tp=0, model=model, X_comp = np.divide(X_comp,0b1010), debug=False):
    x = (x).astype(int)
    model.eval()
    X = make_X(x)
    preds = model(pt_ft(i2t(X.T[None])))
    idx = preds.argmax().item()
    
    if given_idx:
        idx = given_idx
    expected = ((25.5-X_comp)).flatten()
    actual_square = get_cropped(X, idx)
    actual = actual_square.flatten().astype(int)
    if debug:
        print({"idx":idx})
        plt.imshow(actual_square)
        plt.show()
        for i in range(9):
            tmp = get_cropped(X, i)
            print(i, get_ij(i), calc_score(tmp))
    sse = ((actual-(expected*10).astype(int))**2).sum()
    redness = (actual_square[:,:,0][actual_square[:,:,0]>230]**2).sum()+1
    if debug:
        print('sse, ', sse, redness)
    if idx==8:
        return (1+preds.max().item())*sse*redness
    return sse*redness

def get_submittable(x):
    return ",".join([str(int(n)) for n in x])

cfn = lambda x: [f"background-color: RGB({x.R},{x.G},{x.B})"] * 3
def show_colors(ans):
    ans = [int(x) for x in ans.split(",")]
    squares = pd.DataFrame(np.array(ans[:27]).reshape([9,3]), columns=["R","G","B"]).T.style.apply(cfn, axis=0)
    circles = pd.DataFrame(np.array(ans[27:]).reshape([9,3]), columns=["R","G","B"]).T.style.apply(cfn, axis=0)
    display(squares)
    display(circles)

In [ ]:
score_best = 1e18
X_best = np.array([[[210, 182, 109],
  [225, 161, 111],
  [208, 184, 109],
  [226, 161, 111],
  [209, 183, 109]],
 [[225, 161, 111],
  [228, 148, 112],
  [224, 163, 111],
  [228, 147, 112],
  [225, 162, 111]],
 [[208, 184, 109],
  [224, 163, 111],
  [207, 185, 108],
  [225, 162, 111],
  [208, 184, 109]],
 [[226, 161, 111],
  [228, 147, 112],
  [225, 162, 111],
  [228, 147, 112],
  [225, 161, 111]],
 [[209, 183, 109],
  [225, 162, 111],
  [208, 184, 109],
  [225, 161, 111],
  [208, 183, 109]]])
for _ in range(4000):
    num = 1
    ili = []
    jli = []
    kli = []
    tmpli = []
    for nu in range(num):
        i = random.randint(0, 4)
        j = random.randint(0, 4)
        k = random.randint(0, 2)
        ili.append(i)
        jli.append(j)
        kli.append(k)
        has = 0
        for tmp in tmpli:
            if i == tmp[0] and j == tmp[1] and k == tmp[2]:
                has = 1
        if has:
            continue
        tmpli.append((i,j,k,X_best[i,j,k]))
        value = random.randint(0, 255)
        X_best[i, j, k] = value
    X_large = resize(X_best)
    im = np.array(get_submittable(X_best.flatten()).split(","))
    sco = score(im, 4)
    if _ % 1000 == 0:
        score(im, 4, debug=True)
        print(_, tmpli, score_best, sco)
    if sco <= score_best:
        score_best = sco
    else:
        for tmp in tmpli:
            i,j,k,value = tmp
            X_best[i, j, k] = value
print(X_best.tolist(), score_best)

In [ ]:
X_best.tolist()

In [ ]:
## then let model chooses idx == 4, use Hill Climbing Algorithm
## sorry the code can't be found